# Temperature calibration - Lakeshore temperature controller
Perform a temperature calibration based on a continously measured temperature scan using the Lakeshore temperature controller.  
An __external thermocouple__ should be positioned at the sample position and connected to the __Lakeshore ch D__.  
The __heatgun nozzle thermocouple__ should be connected to the __Lakeshore ch C__.

In [ ]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'size':'large'})

In [ ]:
fname = DM.findScan()
# user-specified target temperatures (K)
target_T = [300,400,500,600,700]

# read data form the .h5 file
meta = DM.getMetaDic(fname)
T_nozzle = meta['lakeshore_tempC']
T_sample = meta['lakeshore_tempD']

# polynomial fit
for deg in range(1,4):
   p = {'{deg}':np.polyfit(T_sample,T_nozzle,deg) for deg in range(1,4)}

# plot the calibration
plt.figure()
plt.title(DM.getScan_id(fname))
plt.xlabel('Sample temperature (K)')
plt.ylabel('Nozzle temperature (K)')
plt.plot([T_nozzle.min(),T_nozzle.min()],[T_nozzle.max(),T_nozzle.max()],'k--',lw=1)
plt.plot(T_sample,T_nozzle,label='Measured')

for deg in p:
    fit = np.polyval(p[deg],T_sample)
    eq = 'y = '+' '.join([f'{c:+.4f} 'f'x^{i}' for i,c in enumerate(p[deg][::-1])]).replace('x^0 ','')
    print(eq)
    plt.plot(T_sample,fit,'--',label=f'poly deg: {deg}')

# print calibration table
print('             Polynomial degree')
print(f'Target (K) : '+' : '.join([f'{deg:^6s}' for deg in p]))
for T in target_T:
    print(f'{T:10.1f} : ' + ' : '.join([f'{np.polyval(p[deg],T):06.1f}' for deg in p]))